In [62]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor

pd.options.display.max_rows = 20

In [11]:
df_csv = pd.read_csv('suumo_tokyo.csv', index_col='index').reset_index(drop=True)

In [13]:
df_csv[:5]

,マンション名,住所,区,市町村,間取り,間取りDK,間取りK,間取りL,間取りS,築年数,...,徒歩2,路線3,駅3,徒歩3,賃料,管理費,敷金,礼金,保証金,敷引償却
0,ＪＲ山手線 上野駅 3階建 築18年,東京都台東区根岸３,台東区,根岸３,2,1,1,1,0,18,...,4.0,東京メトロ日比谷線,入谷駅,6.0,133000.0,10000,133000.0,0.0,0.0,0.0
1,ＪＲ山手線 上野駅 3階建 築18年,東京都台東区根岸３,台東区,根岸３,2,1,1,1,0,18,...,4.0,東京メトロ日比谷線,入谷駅,6.0,138000.0,10000,138000.0,0.0,0.0,0.0
2,ＪＲ山手線 上野駅 3階建 築18年,東京都台東区根岸３,台東区,根岸３,2,1,1,1,0,18,...,4.0,東京メトロ日比谷線,入谷駅,6.0,138000.0,11000,138000.0,0.0,0.0,0.0
3,ＪＲ山手線 上野駅 3階建 築18年,東京都台東区根岸３,台東区,根岸３,2,1,1,1,0,18,...,4.0,東京メトロ日比谷線,入谷駅,6.0,138000.0,12000,138000.0,0.0,0.0,0.0
4,パークアクシス清澄白河,東京都江東区扇橋１,江東区,扇橋１,1,0,1,0,0,11,...,11.0,都営新宿線,菊川駅,12.0,84000.0,8000,84000.0,84000.0,0.0,0.0


In [15]:
list(df_csv.columns)

['マンション名',
 '住所',
 '区',
 '市町村',
 '間取り',
 '間取りDK',
 '間取りK',
 '間取りL',
 '間取りS',
 '築年数',
 '建物高さ',
 '階1',
 '専有面積',
 '賃料+管理費',
 '敷/礼/保証金',
 '路線1',
 '駅1',
 '徒歩1',
 '路線2',
 '駅2',
 '徒歩2',
 '路線3',
 '駅3',
 '徒歩3',
 '賃料',
 '管理費',
 '敷金',
 '礼金',
 '保証金',
 '敷引償却']

In [24]:
df_csv[['階1']][:20].T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
階1,1.0,1.0,3.0,3.0,1.0,1.0,5.0,6.0,7.0,2.0,10.0,7.0,7.0,29.0,30.0,16.0,13.0,41.0,2.0,5.0


In [40]:
df_all = df_csv.copy()

In [41]:
# Rename columns
df_all.columns = ["name", "address", "ward", "city", "rooms", "DK", "K", "L", "S", "age", "height", "floor", "area",
                  "rent_mng", "initial_cost", "route1", "station1", "distance1", "route2", "station2", "distance2",
                  "route3", "station3", "distance3", "rent", "mng_fee", "deposit", "gratuity", "security",
                  "non_refund_depreciation"]

In [55]:
rent_upper_threshold = 200000
df = df_all.copy().query('rent_mng <= %d' % rent_upper_threshold).reset_index(drop=True)
df[:5]

,name,address,ward,city,rooms,DK,K,L,S,age,...,distance2,route3,station3,distance3,rent,mng_fee,deposit,gratuity,security,non_refund_depreciation
0,ＪＲ山手線 上野駅 3階建 築18年,東京都台東区根岸３,台東区,根岸３,2,1,1,1,0,18,...,4.0,東京メトロ日比谷線,入谷駅,6.0,133000.0,10000,133000.0,0.0,0.0,0.0
1,ＪＲ山手線 上野駅 3階建 築18年,東京都台東区根岸３,台東区,根岸３,2,1,1,1,0,18,...,4.0,東京メトロ日比谷線,入谷駅,6.0,138000.0,10000,138000.0,0.0,0.0,0.0
2,ＪＲ山手線 上野駅 3階建 築18年,東京都台東区根岸３,台東区,根岸３,2,1,1,1,0,18,...,4.0,東京メトロ日比谷線,入谷駅,6.0,138000.0,11000,138000.0,0.0,0.0,0.0
3,ＪＲ山手線 上野駅 3階建 築18年,東京都台東区根岸３,台東区,根岸３,2,1,1,1,0,18,...,4.0,東京メトロ日比谷線,入谷駅,6.0,138000.0,12000,138000.0,0.0,0.0,0.0
4,パークアクシス清澄白河,東京都江東区扇橋１,江東区,扇橋１,1,0,1,0,0,11,...,11.0,都営新宿線,菊川駅,12.0,84000.0,8000,84000.0,84000.0,0.0,0.0


In [69]:
labels_to_drop = ['name', 'address', 'rent_mng', 'initial_cost', 'route1', 'station1', 'route2', 'station2',
                  'distance2', 'route3', 'station3', 'distance3', 'rent', 'mng_fee', 'deposit', 'gratuity',
                  'security', 'non_refund_depreciation']
%time X = pd.get_dummies(df.drop(labels=labels_to_drop, axis=1))
y = df.rent_mng

CPU times: user 1min 1s, sys: 2.54 s, total: 1min 4s
Wall time: 22.1 s


In [70]:
X[:5]

,rooms,DK,K,L,S,age,height,floor,area,distance1,...,city_麹町２,city_麹町３,city_麹町５,city_麻布十番１,city_麻布十番２,city_麻布十番３,city_麻布十番４,city_麻布台１,city_麻布台３,city_麻布狸穴町
0,2,1,1,1,0,18,3,1.0,55.69,14.0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,1,1,0,18,3,1.0,55.69,14.0,...,0,0,0,0,0,0,0,0,0,0
2,2,1,1,1,0,18,3,3.0,55.69,14.0,...,0,0,0,0,0,0,0,0,0,0
3,2,1,1,1,0,18,3,3.0,55.69,14.0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,1,0,0,11,8,1.0,25.93,11.0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
regr = RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=0, verbose=1)

In [72]:
%time regr.fit(np.nan_to_num(X), y)

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  2.3min finished


CPU times: user 8min 24s, sys: 49 s, total: 9min 13s
Wall time: 3min 47s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=0, verbose=1, warm_start=False)